https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb#Timestamped-GeoJSON

In [1]:
import pandas as pd
import folium
from folium import plugins
import requests

pd.options.display.max_columns = None

In [2]:
# df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/'
#                  'feed/v1.0/summary/2.5_day.csv')
df = pd.read_csv('https://earthquake.usgs.gov/earthquakes/'
                 'feed/v1.0/summary/2.5_month.csv')

# df['time'] = pd.to_datetime(df['time'], utc=True)

df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2019-05-29T19:14:36.010Z,38.809666,-122.817833,2.49,2.86,md,40.0,44.0,NaN,0.06,nc,nc73189226,2019-05-29T20:36:03.510Z,"6km NW of The Geysers, CA",earthquake,0.17,0.3,0.150,25.0,automatic,nc,nc
1,2019-05-29T18:40:43.285Z,40.457000,63.618200,10.00,4.70,mb,NaN,125.0,4.536,0.82,us,us70003stv,2019-05-29T20:40:30.807Z,"38km NNE of Gazli, Uzbekistan",earthquake,9.30,2.0,0.066,70.0,reviewed,us,us
2,2019-05-29T18:33:28.072Z,-21.118200,170.079900,80.67,4.90,mb,NaN,91.0,2.862,1.34,us,us70003stq,2019-05-29T18:54:31.040Z,"193km SSE of Isangel, Vanuatu",earthquake,11.90,7.6,0.070,64.0,reviewed,us,us
3,2019-05-29T18:21:48.343Z,-31.773500,-178.804700,46.49,5.00,mb,NaN,75.0,2.631,1.23,us,us70003stl,2019-05-29T18:41:53.040Z,"38km SSE of L'Esperance Rock, New Zealand",earthquake,10.40,8.2,0.068,72.0,reviewed,us,us
4,2019-05-29T16:44:46.492Z,58.899700,-154.390600,116.40,2.50,ml,NaN,NaN,NaN,0.76,ak,ak0196uov6c3,2019-05-29T18:52:20.040Z,"99km SSE of Old Iliamna, Alaska",earthquake,NaN,0.5,NaN,NaN,automatic,ak,ak


In [3]:
# get faults
r = requests.get('https://raw.githubusercontent.com/'
                 'fraxen/tectonicplates/master/GeoJSON/'
                 'PB2002_boundaries.json')

fault_features = r.json()['features']

# for feature in fault_features:
#     feature['properties']['color'] = 'red'

In [43]:
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [r['longitude'], r['latitude']],
        },
        'properties': {
            'time': r['time'][0:-1],
            'popup': (
                f"<strong>Time:</strong> {r['time']}<br>"
                f"<strong>Place:</strong> {r['place']}<br>"
                f"<strong>Magnitude:</strong> {r['mag']} {r['magType']}<br>"
                f"<strong>Depth:</strong> {r['depth']}<br>"
            ),
            'icon': 'circle',
            'iconstyle': {
                'fillOpacity': 0.5,
                'stroke': 0,
                'radius': r['mag'] * 2.5
            },
        }
    } for i, r in df.iterrows()
]

m = folium.Map(
    tiles='CartoDBpositron',
#     zoom_start=1,
#     no_wrap=True,
    min_zoom=1.5,
    max_zoom=5,
)

# add faults
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': fault_features,
    },
    style_function = lambda x: {
        'color': 'red',
        'weight': 0.5,
    }
).add_to(m)

plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='PT6H', # six hour
    time_slider_drag_update=True,
    duration='PT12H',
    date_options='YYYY-MM-DD HH UTC'
).add_to(m)

folium.plugins.Fullscreen(
    position='topright',
    force_separate_button=True,
).add_to(m)

m.save('earthquakes.html')
m

In [39]:
# from IPython.display import IFrame
# IFrame('earthquakes_2.5_day.html', width='100%', height=500)